In [1]:
import pandas as pd

In [24]:
data = pd.DataFrame.from_csv('./uci-news-aggregator.csv',
                           #sep=',',
                           #names='ID TITLE URL PUBLISHER CATEGORY STORY HOSTNAME TIMESTAMP'.split(' '),
                           #low_memory=False,
                           #indexer='ID'
                  )
data.head()

,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
ID,,,,,,,
1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


по идее нам нужны только названия и категории. Сформируем нужный датасет

In [27]:
classification_data = data[['TITLE','CATEGORY']]
classification_data.groupby('CATEGORY').describe()

TITLE
CATEGORY                                                          
b        count                                              115967
         unique                                             111903
         top                                   Posted by Imaduddin
         freq                                                   52
e        count                                              152469
         unique                                             146952
         top     The article requested cannot be found! Please ...
         freq                                                  130
m        count                                               45639
         unique                                              43719
         top                                        Share this on:
         freq                                                    9
t        count                                              108344
         unique                                             104733
         top                                         Business Wire
         freq                                                   29

видно, что всего 4 класса и они, в принципе, не очень разбаллансированы. Выберем лучшую векторизацию.

In [33]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(classification_data, test_size=0.2, random_state=12)

In [34]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
import re

st = LancasterStemmer()

lmtzr = WordNetLemmatizer()
tknzr = TweetTokenizer()
punct = re.compile(',|\.|;|:|\?|\)|\(|!')
non_letters = re.compile('[^a-zA-Z]')
stops = set(stopwords.words("english"))

def tokenize_np_nl_ds(text):
    #no punctuation, no lemmatisation, delete stops
    letters_only = non_letters.sub(" ", text) 
    words = letters_only.lower().split()                 
    meaningful_words = [w for w in words if w not in stops]   
    return meaningful_words 

def tokenize_np_nl(text):
    #delete punctuation, no lemmatisation, leave stops
    letters_only = non_letters.sub(" ", text) 
    words = letters_only.lower().split()                   
    return  words 

def tokenize_p_nl_ds(text):
    #leave punct, no lemmatisation, delete stops
    txt = tknzr.tokenize(text)
    meaningful_words = [w for w in txt if w not in stops]   
    return meaningful_words 

def tokenize_p_nl(text):
    #leave punct, no lemmatisation, leave stops
    txt = tknzr.tokenize(text)
    return  txt

def tokenize_p_l_ds(text):
    #leave punct, lemmetize, delete stops
    txt = tknzr.tokenize(text)
    meaningful_words = [lmtzr.lemmatize(w) for w in txt if w not in stops]   
    return  meaningful_words 

def tokenize_p_l(text):
    #leave punct, lemmetize, leave stops
    txt = tknzr.tokenize(text) 
    return [lmtzr.lemmatize(w) for w in txt]

def tokenize_np_l_ds(text):
    #delete punct, lemmetize, delete stops
    txt = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = txt.lower().split()
    return  [lmtzr.lemmatize(w) for w in words if w not in stops]

def tokenize_np_l(text):
    #delete punct, lemmetize, delete stops
    txt = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = txt.lower().split()
    return  [lmtzr.lemmatize(w) for w in words]



#same for stemmatisation
def tokenize_p_s_ds(text):
    #leave punct, lemmetize, delete stops
    txt = tknzr.tokenize(text)
    meaningful_words = [st.stem(w) for w in txt if w not in stops]   
    return  meaningful_words 

def tokenize_p_s(text):
    #leave punct, lemmetize, leave stops
    txt = tknzr.tokenize(text) 
    return [st.stem(w) for w in txt]

def tokenize_np_s_ds(text):
    #delete punct, lemmetize, delete stops
    txt = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = txt.lower().split()
    return  [st.stem(w) for w in words if w not in stops]

def tokenize_np_s(text):
    #delete punct, lemmetize, delete stops
    txt = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = txt.lower().split()
    return  [st.stem(w) for w in words]

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

parametrs = [tokenize_np_nl_ds,
            tokenize_np_nl,
            tokenize_p_nl_ds,
            tokenize_p_nl,
            tokenize_p_l_ds,
            tokenize_p_l,
            tokenize_np_l_ds,
            tokenize_np_l,
            tokenize_p_s_ds,
            tokenize_p_s,
            tokenize_np_s_ds,
            tokenize_np_s,
            ]

clf = MultinomialNB()
all_params = {}
for param in parametrs:
    cv = CountVectorizer(analyzer='word', tokenizer=param)
    cv.fit_transform(classification_data['TITLE'])
    transformed_train = cv.transform(train['TITLE'])
    transformed_test = cv.transform(test['TITLE'])
    clf.fit(transformed_train, train['CATEGORY'])
    print()
    print('For CountVectoriser and for '+ str(param))
    print('accuracy_score is:    '+ str(accuracy_score(test['CATEGORY'], clf.predict(transformed_test))))
    all_params[accuracy_score( clf.predict(transformed_test), test['CATEGORY'])] = str(param) + '  +  count'
    print('------------->  tfidf  <------------')
    
    tcv = TfidfVectorizer(analyzer='word', tokenizer=param)
    tcv.fit_transform(classification_data['TITLE'])
    transformed_train = tcv.transform(train['TITLE'])
    transformed_test = tcv.transform(test['TITLE'])
    clf.fit(transformed_train, train['CATEGORY'])
    print('accuracy_score is:    '+ str(accuracy_score( test['CATEGORY'], clf.predict(transformed_test))))
    all_params[accuracy_score( clf.predict(transformed_test), test['CATEGORY'])] = str(param) + '  +  tfidf'


For CountVectoriser and for <function tokenize_np_nl_ds at 0x112fb7bf8>
accuracy_score is:    0.927110458785
------------->  tfidf  <------------
accuracy_score is:    0.92548884996

For CountVectoriser and for <function tokenize_np_nl at 0x112fb79d8>
accuracy_score is:    0.924672127267
------------->  tfidf  <------------
accuracy_score is:    0.922908479712

For CountVectoriser and for <function tokenize_p_nl_ds at 0x112fb7c80>
accuracy_score is:    0.92688556413
------------->  tfidf  <------------
accuracy_score is:    0.925169262819

For CountVectoriser and for <function tokenize_p_nl at 0x112fb7b70>
accuracy_score is:    0.924423559491
------------->  tfidf  <------------
accuracy_score is:    0.922979499077

For CountVectoriser and for <function tokenize_p_l_ds at 0x112fb7d90>
accuracy_score is:    0.924766819753
------------->  tfidf  <------------
accuracy_score is:    0.923062355002

For CountVectoriser and for <function tokenize_p_l at 0x112fb7e18>
accuracy_score is:    0.

In [37]:
print('The best combo:')
print(all_params[sorted(all_params)[len(all_params)-1]])

The best combo:
<function tokenize_np_nl_ds at 0x112fb7bf8>  +  count


построим бейзлайн.
Для этого еще раз разделим на train и test, по которым мы будем проверять модели

In [41]:
to_train, to_test = train_test_split(train, test_size=0.3, random_state=15)

In [42]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

cv = CountVectorizer(analyzer='word', tokenizer=tokenize_np_nl_ds)
cv.fit_transform(classification_data['TITLE'])
transformed_to_train = cv.transform(to_train['TITLE'])
transformed_to_test = cv.transform(to_test['TITLE'])
transformed_test = cv.transform(test['TITLE'])

dc = DummyClassifier()
dc.fit(transformed_to_train, to_train['CATEGORY'])
#print(to_test['Line'].tolist())
print(classification_report(to_test['CATEGORY'], dc.predict(transformed_to_test), target_names=['b', 'e', 'm' , 't']))

             precision    recall  f1-score   support

          b       0.28      0.28      0.28     27882
          e       0.36      0.36      0.36     36525
          m       0.11      0.11      0.11     10943
          t       0.26      0.26      0.26     26031

avg / total       0.29      0.29      0.29    101381



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

params = [{'penalty':['l1','l2'], 'C':[0.1, 0.5, 1.0, 2, 5]}]

lr = LogisticRegression()#solver='sag', multi_class='multinomial', n_jobs=-1, max_iter=100)

clf = GridSearchCV(lr, params)
clf.fit(transformed_to_train, to_train['CATEGORY']) 
print(clf.best_params_, clf.best_score_)

print(classification_report(to_test['CATEGORY'], clf.predict(transformed_to_test), target_names=['b', 'e', 'm' , 't']))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

mat = confusion_matrix(to_test['CATEGORY'], clf.predict(transformed_to_test))
plt.figure()
plot_confusion_matrix(mat, classes=['b', 'e', 'm' , 't'],
                      title='Confusion matrix. Logistic Regression')
plt.show()